In [ ]:
!pip install hazm

In [ ]:
import hazm
from hazm import *
import pprint
from hazm import stopwords_list

In [ ]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
n=-1
df = pd.read_csv(r'Varzeshi.csv')
test_Column = df['Text']
for x in test_Column:
    n = n + 1
    text = x

    #Tokenizing and Removing Stopwords using hazm
    sentence_tokens = []
    normalizer = hazm.Normalizer()
    sentences=hazm.sent_tokenize(normalizer.normalize(text))
    sentence_tokens_included_stopwords=[hazm.word_tokenize(sentence) for sentence in sentences]
    for sent in sentence_tokens_included_stopwords:
      for w in sent:
        if w in stopwords_list():
          sent.remove(w)

    sentence_tokens = sentence_tokens_included_stopwords

    #Embedding with word2vec
    w2v=Word2Vec(sentence_tokens,vector_size=1,min_count=1,epochs=1000)
    sentence_embeddings=[[w2v.wv[word][0] for word in words] for words in sentence_tokens]
    max_len=max([len(tokens) for tokens in sentence_tokens])
    sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]

    #Calculating Simularity using Cosine Simularity
    similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
    for i,row_embedding in enumerate(sentence_embeddings):
        for j,column_embedding in enumerate(sentence_embeddings):
            similarity_matrix[i][j]=abs(1-spatial.distance.cosine(row_embedding,column_embedding))

    #Creating Graph
    nx_graph = nx.from_numpy_array(similarity_matrix)
    scores = nx.pagerank(nx_graph)
    top_sentence={sentence:scores[index] for index,sentence in enumerate(sentences)}
    top=dict(sorted(top_sentence.items(), key=lambda x: x[1], reverse=True)[:4])
    print("<<<text " + str(n+1) + " summarized>>>")
    temp_summary_list = []
    for sent in sentences:
        if sent in top.keys():
          temp_summary_list.append(sent)
    summary=str(temp_summary_list)
    summary=summary.replace("[","")
    summary=summary.replace("]","")
    summary=summary.replace(","," ")
    summary=summary.replace("\'","")
    summary=summary.replace("\\u200c"," ")
    df.loc[n, 'Textrank'] = summary
df.to_csv('newVarzeshi.csv')

df.head()


Streaming output truncated to the last 5000 lines.
<<<text 1497 summarized>>>
<<<text 1498 summarized>>>
<<<text 1499 summarized>>>
<<<text 1500 summarized>>>
<<<text 1501 summarized>>>
<<<text 1502 summarized>>>
<<<text 1503 summarized>>>
<<<text 1504 summarized>>>
<<<text 1505 summarized>>>
<<<text 1506 summarized>>>
<<<text 1507 summarized>>>
<<<text 1508 summarized>>>
<<<text 1509 summarized>>>
<<<text 1510 summarized>>>
<<<text 1511 summarized>>>
<<<text 1512 summarized>>>
<<<text 1513 summarized>>>
<<<text 1514 summarized>>>
<<<text 1515 summarized>>>
<<<text 1516 summarized>>>
<<<text 1517 summarized>>>
<<<text 1518 summarized>>>
<<<text 1519 summarized>>>
<<<text 1520 summarized>>>
<<<text 1521 summarized>>>
<<<text 1522 summarized>>>
<<<text 1523 summarized>>>
<<<text 1524 summarized>>>
<<<text 1525 summarized>>>
<<<text 1526 summarized>>>
<<<text 1527 summarized>>>
<<<text 1528 summarized>>>
<<<text 1529 summarized>>>
<<<text 1530 summarized>>>
<<<text 1531 summarized>>>
<<<t

,Unnamed: 0.1,Unnamed: 0,ID,Summary,Text,Category,LengthText,Textrank
0,0,6,1000011,سرمربی تیم ملی فوتبال کامبوج گفت: ایران تیمی پ...,به گزارش مشرق، فلیکس دالماس در نشست خبری پیش ا...,ورزش,2.0,البته من وقت بیشتری نسبت به هوندا دارم ولی او ...
1,1,8,1000013,بازیکن تیم ملی فوتبال کامبوج گفت: می‌خواهیم به...,به گزارش مشرق، سو ئی ویسا بازیکن تیم ملی کامبو...,ورزش,0.0,به گزارش مشرق، سو ئی ویسا بازیکن تیم ملی کامبو...
2,2,12,1000018,ورزشگاه آزادی در فاصله ۲۴ ساعت مانده به بازی ت...,به گزارش مشرق، تیم‌های ملی فوتبال ایران و کامب...,ورزش,1.0,به گزارش مشرق، تیم های ملی فوتبال ایران و کامب...
3,3,21,1000028,هنوز لیگ به هفته های میانی نیم فصل اول نرسیده ...,سرویس ورزش مشرق – در پایان هفته ششم لیگ برتر، ...,ورزش,2.0,نساجی که با نظر مهاجری بسته شده و چشم امید بخش...
4,4,31,1000039,آخرین تمرین تیم ملی فوتبال کامبوج در شرایطی بر...,به گزارش مشرق، آخرین تمرین تیم ملی کامبوج برای...,ورزش,1.0,به گزارش مشرق، آخرین تمرین تیم ملی کامبوج برای...
